In [1]:
import gograph
import similarity
import importlib
import re
import time
import annotations

import pandas as pd
import numpy as np

In [2]:
def reload_my_modules():
    """Re-import my modules into the IPython shell"""
    importlib.reload(annotations)
    importlib.reload(gograph)
    importlib.reload(similarity)

In [3]:
%%time
reload_my_modules()
fp = 'data/goa_human.gaf'
anno = annotations.Annotations(fp)

CPU times: user 1.1 s, sys: 97.5 ms, total: 1.19 s
Wall time: 1.3 s


/home/xyz/.local/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


In [4]:
# num proteins
print(anno.annotations.groupby('DB_Object_ID').size().count())
# num annos per protein
print(anno.annotations.groupby('DB_Object_ID').size().mean())
#
print(anno.annotations.groupby('DB_Object_ID').size().quantile([0, 0.25, 0.5, 0.75, 1]))
# total annos
print(len(anno.annotations))

19735
25.612515834811248
0.00      1.0
0.25      7.0
0.50     15.0
0.75     30.0
1.00    914.0
dtype: float64
505463


In [5]:
%%time
reload_my_modules()
fp = 'data/go-basic.obo'
#fp = 'data/gene_ontology_edit.obo.2019-07-01'
ontology = gograph.OBOParser(fp).parse_ontology()

CPU times: user 1.72 s, sys: 52.5 ms, total: 1.78 s
Wall time: 1.79 s


In [6]:
t = time.time()
ontology.make_ancestry_matrix()
print(time.time()-t)

7.330615758895874


In [7]:
ontology.ancestry_matrix.matrix.setdiag(1)

In [8]:
anno.annotations = anno.annotations.loc[anno.annotations.Aspect == 'P', :]

In [9]:
len(anno.annotations)

165155

In [10]:
importlib.reload(annotations)
ft = annotations.FrequencyTable(anno, ontology.ancestry_matrix)

/home/xyz/Documents/CodeRepos/ggid/annotations.py:52: RuntimeWarning: divide by zero encountered in log10
  self.information_content = -1*np.log10(self.deep_count.sum(axis=0)/self.deep_count.sum())


In [11]:
ft.ic['GO:0008150']
#'GO:0008152'
#'GO:0009987'

1.3485159476242856

In [12]:
ft.index_dict['GO:0120077']

38807

In [13]:
proteins = list(set(list(anno.annotations.DB_Object_Symbol)))
print(len(proteins))

17936


In [14]:
len(set(anno.annotations.GO_ID))

12369

In [17]:
reload_my_modules()

#P25098 GRK2
#P35626 GRK3

sim = similarity.Similarity(anno, ontology, ft, list(p.uniprot), namespace='P', verbose=True)

512


In [18]:
###old time (10)
###11.857779264450073

In [19]:
sim.calculate_similarity_mult_cpu()

worker started
worker started


In [16]:
p = pd.read_csv('data/list_of_human_kinases.csv')
p.head()

,Unnamed: 0,gene_symbol,uniprot
0,0,AKT1,P31749
1,1,AKT2,P31751
2,2,AKT3,Q9Y243
3,3,MRCKA,Q5VT25
4,4,MRCKB,Q9Y5S2


In [54]:
anno.annotations[anno.annotations.DB_Object_ID == 'Q5VT25']

,DB,DB_Object_ID,DB_Object_Symbol,Qualifier,GO_ID,DB_Reference,Evidence_Code,With_or_From,Aspect,DB_Object_Name,DB_Object_Synonym,DB_Object_Type,Taxon,Date,Assigned_By,Annotation_Extension,Gene_Product_Form_ID
281025,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0006468,PMID:9092543,IDA,NaN,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20051004,UniProt,NaN,NaN
281026,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0016477,PMID:18854160,IMP,NaN,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20110919,UniProt,NaN,NaN
281028,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0031032,PMID:18854160,IMP,NaN,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20110919,UniProt,NaN,NaN
281030,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0031532,PMID:9418861,IDA,NaN,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20051004,UniProt,NaN,NaN
281031,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0035556,GO_REF:0000002,IEA,InterPro:IPR002219,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20200222,InterPro,NaN,NaN
453930,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0031032,PMID:21873635,IBA,FB:FBgn0002466|FB:FBgn0003149|FB:FBgn0026181|P...,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20190308,GO_Central,NaN,NaN
498416,UniProtKB,Q5VT25,CDC42BPA,NaN,GO:0018107,PMID:21873635,IBA,MGI:MGI:107926|PANTHER:PTN002743840|UniProtKB:...,P,Serine/threonine-protein kinase MRCK alpha,CDC42BPA|KIAA0451,protein,taxon:9606,20190308,GO_Central,NaN,NaN


In [ ]:
print("hi")

In [ ]:
print(sim.workers)

In [83]:
t0 = time.time()
sim.calculate_similarity()
print(time.time() - t0)
print(sim.counter)

58.02841639518738
63839231


In [84]:
len(sim.sim_matrix.row)



131328

In [85]:
print(sim.sim_matrix.nnz)

131328


In [86]:
sim.sim_matrix[0,1]

TypeError: 'coo_matrix' object is not subscriptable

In [88]:
sim.sim_matrix.todense()[0,0]

2.099084886160813

In [91]:
smt = sim.sim_matrix.todense()

In [100]:
smt

matrix([[2.09908489, 1.92136692, 1.86507187, ..., 1.89110765, 1.837878  ,
         1.72123156],
        [0.        , 2.07676028, 1.84519344, ..., 1.8458094 , 1.84945615,
         1.71754962],
        [0.        , 0.        , 2.28748771, ..., 1.85830777, 1.85698712,
         1.75924367],
        ...,
        [0.        , 0.        , 0.        , ..., 1.93452153, 1.79996354,
         1.70440276],
        [0.        , 0.        , 0.        , ..., 0.        , 1.92693684,
         1.71532161],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         1.7710692 ]])

In [99]:
smt.T

matrix([[2.09908489, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [1.92136692, 2.07676028, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [1.86507187, 1.84519344, 2.28748771, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [1.89110765, 1.8458094 , 1.85830777, ..., 1.93452153, 0.        ,
         0.        ],
        [1.837878  , 1.84945615, 1.85698712, ..., 1.79996354, 1.92693684,
         0.        ],
        [1.72123156, 1.71754962, 1.75924367, ..., 1.70440276, 1.71532161,
         1.7710692 ]])

In [169]:
from scipy import sparse

In [105]:
smt2=smt+smt.T

In [171]:
smt3 = sparse.coo_matrix((a + a.T))

In [176]:
smt3.col
smt3.row

array([0, 0, 1, 1, 2, 3, 3, 4], dtype=int32)

In [157]:
a = np.random.rand(5,5)
mask = a.argsort(axis=0)>3#[-3:]

In [161]:
a[mask] = 10
a[~mask] = 0

In [147]:
np.argsort?

In [103]:
np.fill_diagonal(smt, 0)

In [137]:
exp = np.random.rand(10,2)

In [139]:
#fig=
px.scatter(x=exp[:,0], y=exp[:,1])
#fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([0.14038433, 0.59935527, 0.81956797, 0.06767122, 0.17242636, 0.67185875,
                          0.00670353, 0.85919367, 0.86002442, 0.94523113]),
              'xaxis': 'x',
              'y': array([0.98655776, 0.80606852, 0.94073389, 0.7190118 , 0.52681298, 0.39335302,
                          0.53521282, 0.92005128, 0.38786813, 0.70485478]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})

In [131]:
54*((1000/500)**2)

216.0

In [ ]:
sim.get_ic_mica('GO:0006807', 'GO:0006807')

In [ ]:
import copy
sim2 = copy.copy(sim)
sim3 = copy.copy(sim)
sim4 = copy.copy(sim)
sim5 = copy.copy(sim)

In [ ]:
print(id(sim))
print(id(sim2))
print(id(sim3))

In [ ]:
import multiprocessing

def function_inside_worker():
    print("hi")

def twoplustwo(x):
    return 2+2

def worker(x):
    """thread worker function"""
    print("worker started")
    print(twoplustwo(x))
    print(len(x))
    print("worker exited")
    #print(id(x))


#print(time.time()-t0)



In [ ]:
divmod(1999, 3)

In [ ]:
ll = list(range(1355))

cpus = 4
a,b = divmod(len(ll),cpus)
print(a, b)

workers = []
for i in range(cpus):
    segment = ll[i*a:(i*a+a)]
    if i == cpus-1:
        segment = ll[i*a:]
    print(len(segment))
    p = multiprocessing.Process(target=worker, args=(segment,))
    workers.append(p)

for p in workers:
    p.start()

In [ ]:
ll[0:2]

In [ ]:
print(id(sim4.ic_mica))
print(id(sim5.ic_mica))

In [ ]:
import multiprocessing
import time

def worker():
    name = multiprocessing.current_process().name
    print(name, 'Starting')
    time.sleep(2)
    print(name, 'Exiting')

def my_service():
    name = multiprocessing.current_process().name
    print(name, 'Starting')
    time.sleep(3)
    print(name, 'Exiting')

service = multiprocessing.Process(name='my_service', target=my_service)
worker_1 = multiprocessing.Process(name='worker 1', target=worker)
worker_2 = multiprocessing.Process(target=worker) # use default name

worker_1.start()
worker_2.start()
service.start()